In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from rascaline import SoapPowerSpectrum, SphericalExpansion
from utils.models.soap import compute_power_spectrum
import ase.io
import numpy as np
import copy
from equistore import Labels, TensorBlock, TensorMap

In [3]:
frames = ase.io.read("data/molecule_conformers_dftb.xyz", ":2")
frames[0].get_chemical_symbols()

['O', 'C', 'C', 'C', 'H', 'H', 'H', 'H', 'H', 'H']

In [4]:
def selected_samples(frames, hypers):
    """Create a list of selected samples so that all blocks have the same set 
    of samples for simpler implementation of compute_power_spectrum"""
    if not isinstance(frames, list):
        frames = [frames]
        
    all_species = set()
    for frame in frames:
        all_species.update(frame.numbers)

    keys = Labels(
        names=["spherical_harmonics_l", "species_center", "species_neighbor"],
        values=np.array([
            [l, s_c, s_n] 
            for l in range(hypers["max_angular"] + 1)
            for s_c in all_species
            for s_n in all_species
        ], dtype=np.int32)
    )
    blocks = []
    for _ in range(len(keys)):
        samples = Labels(
            names=["structure", "center"],
            values=np.array([
                [s, c] 
                for s in range(len(frames))
                for c in range(len(frames[s]))
            ], dtype=np.int32)
        )
        
        blocks.append(TensorBlock(
            values=np.empty((len(samples), 1)),
            samples=samples, 
            components=[],
            properties=Labels.single()
        ))
        
    return TensorMap(keys, blocks)

In [6]:
HYPER_PARAMETERS = {
    "cutoff": 3,
    "max_radial": 6,
    "max_angular": 4,
    "atomic_gaussian_width": 0.3,
    "center_atom_weight": 1.0,
    "radial_basis": {
        "Gto": {},
    },
    "cutoff_function": {
        "ShiftedCosine": {"width": 0.5},
    },
}

calculator = SphericalExpansion(**HYPER_PARAMETERS)
descriptor = calculator.compute(frames,
                                selected_samples=selected_samples(frames, HYPER_PARAMETERS))
ps = compute_power_spectrum(descriptor)
print(f"ps has {len(ps.blocks())} blocks")

ps_rascaline = SoapPowerSpectrum(**HYPER_PARAMETERS).compute(frames)
print(f"ps_rascaline has {len(ps_rascaline.blocks())} blocks")

ps has 18 blocks
ps_rascaline has 18 blocks


/opt/homebrew/lib/python3.10/site-packages/rascaline/systems/ase.py:57: UserWarning: periodic boundary conditions are disabled, but the cell matrix is not zero, we will set the cell to zero.
  warnings.warn(
